# Imports

In [1]:
import numpy as np
import cvxpy as cp
from classes import NpaHierarchy
from itertools import product as iter_product
from Distribution import Distribution

# Bell Scenario Definition

In [48]:
N = 2
NA, NB, NX, NM, = (N,) * 4

bell_scenario=(NA,NB,NX,NM)

In [49]:
A = np.array(range(NA))
X = np.array(range(NX))
B = np.array(range(NB))
M = np.array(range(NM))

# Distribution generator

for this particular one, we pick the optimal non-signaling distribution (not a quantum one) for the CHSH game.
$$
    p(a,b|x,m)= \frac{1}{2}\mathbb{1}_{a\oplus b = x\cdot m}
$$

In [50]:
vals_perfect = np.zeros(shape=(NA,NB,NX,NM))
for a,b,x,m in iter_product(A,B,X,M):
    lhs = (a+b)%2
    rhs = x*m
    val = 0.5*(lhs==rhs)
    vals_perfect[a,b,x,m] = val
p_perfect = Distribution(vals_perfect, bell_scenario)

## Local distributions

In [51]:
vals = np.zeros(shape=(NA,NB,NX,NM))
for a,b,x,m in iter_product(A,B,X,M):
    vals[a,b,x,m] = 0.5*(a==x)*(b==m)+0.5*(a==x)*(b!=m)
    # vals[a,b,x,m] = 0.25
p_local = Distribution(vals, bell_scenario)

## Quantum distributions

In [52]:
omega = np.cos(np.pi/8)**2
vals = np.zeros(shape=(NA,NB,NX,NM))
for a,b,x,m in iter_product(A,B,X,M):
    if x == 2 or m == 2:
        vals[a,b,x,m] = 0
    if x*m:
        vals[a,b,x,m] = 0.5*(a != b)*(2*omega - 1) + 0.5*(1 - omega)
    else:
        vals[a,b,x,m] = 0.5*(a == b)*(2*omega - 1) + 0.5*(1 - omega)
p_quantum = Distribution(vals, bell_scenario)

For the first step of the NPA hierarchy, the $\Gamma$ matrix has the following form.
$$\Gamma\left(\mathcal{P}\right)=\left(\begin{array}{ccccccccc}
\overset{\left(\perp\right)}{\overbrace{1}} & \overset{\left(a=0,x=0\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=1,x=0\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=0,x=1\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=1,x=1\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(b=0,m=0\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=1,m=0\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=0,m=1\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=1,m=1\right)}{\overbrace{\Pr\left(b|m\right)}}\\
 & \Pr\left(a|x\right) & 0 & ? & ? & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  & \Pr\left(a|x\right) & ? & ? & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  & \Pr\left(a|x\right) & 0 & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  &  & \Pr\left(a|x\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  &  &  & \Pr\left(b|m\right) & 0 & ? & ?\\
 &  &  &  &  &  & \Pr\left(b|m\right) & ? & ?\\
 &  &  &  &  &  &  & \Pr\left(b|m\right) & 0\\
 &  &  &  &  &  &  &  & \Pr\left(b|m\right)
\end{array}\right)$$

In [57]:
npa = NpaHierarchy(bell_scenario, 5)

In [54]:
gamma, variables = npa.feasability(p_perfect)
mu_gamma = cp.Variable()

In [55]:
gamma_bmat = cp.bmat(gamma)
I = np.eye(len(gamma), len(gamma))
constraints = list()
# constraints = [variables[key] >= 0 for key in variables]
constraints += [gamma_bmat - mu_gamma*I >> 0]

In [56]:
objective = cp.Maximize(mu_gamma)
problem = cp.Problem(objective, constraints)
problem.solve()


3.02582333168152e-06